**

Positive: happy, good, excellent, amazing, great, wonderful, fantastic, outstanding, superb, positive, strong, confident, promising, robust, solid, optimistic, encouraging, favorable, profit, success, growth, high, gain, rise, up, bullish, increase, peak, record-high, outperform, surge, rally, revenue, earnings, dividends, expansion, upgrade, investment, acquisition, innovation, wealth, prosperity, uptrend, rallying, skyrocketing, surge, breakout, rebound, resurgence, breakthrough, recovery, exceeding expectations, soaring, beating estimates, surpassing forecasts, buy, buy-back, top pick, outperform, bullish trend, strong demand, oversubscribed, buy recommendation, institutional buying, accumulation, positive guidance, share repurchase, capital appreciation, strong economy, low unemployment, surplus, GDP growth, high consumer confidence, tax cuts, market optimism, job creation, rising wages, stable inflation, expansionary policy, fiscal stimulus

**

In [2]:
import os
import time
import random
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode, urlparse, parse_qs
from datetime import datetime, timedelta
from newspaper import Article
import pandas as pd
import yfinance as yf

# Stocks to scrape news for
stocks = [ "SPG", "WELL"
    # "XOM", "CVX", "COP", "SLB", "EOG", "OXY", "VLO", "MPC", "PSX",
    # "DOW", "DD", "LIN", "APD", "SHW", "NEM", "FCX", "ALB", "IP", "ECL",
    # "GE", "HON", "UNP", "BA", "MMM", "CAT", "LMT", "RTX", "NOC", "DE",
    # "AMZN", "TSLA", "HD", "MCD", "NKE", "SBUX", "BKNG", "LOW", "TGT", "GM"
]

GOOGLE_NEWS_URL = "https://www.google.com/search"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
OUTPUT_DIR = "news_articles"
os.makedirs(OUTPUT_DIR, exist_ok=True)

FETCH_LIMIT = 15
MAX_VALID_ARTICLES = 5


def fetch_full_article(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        return ""


def clean_google_url(google_url):
    parsed_url = urlparse(google_url)
    query_params = parse_qs(parsed_url.query)
    return query_params.get("q", [google_url])[0]


def convert_relative_date(relative_date):
    now = datetime.today()
    if "hour" in relative_date:
        num = int(relative_date.split()[0])
        article_date = now - timedelta(hours=num)
    elif "day" in relative_date:
        num = int(relative_date.split()[0])
        article_date = now - timedelta(days=num)
    elif "week" in relative_date:
        num = int(relative_date.split()[0])
        article_date = now - timedelta(weeks=num)
    elif "minute" in relative_date:
        article_date = now
    else:
        article_date = now
    return article_date.strftime("%Y-%m-%d")


def scrape_news(stock):
    print(f"\nFetching news for: {stock}...")
    valid_articles = []
    params = {"q": f"{stock} stock news", "tbm": "nws", "hl": "en", "gl": "us"}
    url = f"{GOOGLE_NEWS_URL}?{urlencode(params)}"
    response = requests.get(url, headers=HEADERS)

    if response.status_code != 200:
        print(f"Failed to fetch news for {stock}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    articles_fetched = 0

    for article in soup.select(".SoaBEf"):
        if articles_fetched >= FETCH_LIMIT:
            break

        title_element = article.select_one(".nDgy9d")
        url_element = article.select_one("a")
        date_element = article.select_one(".OSrXXb")

        title = title_element.text.strip() if title_element else "No Title"
        url = url_element["href"] if url_element else ""
        raw_date = date_element.text.strip() if date_element else "Unknown Date"

        if url.startswith("/url?"):
            url = clean_google_url(url)

        full_article = fetch_full_article(url)
        if not full_article.strip():
            continue

        valid_articles.append({
            "title": title,
            "url": url,
            "date": convert_relative_date(raw_date),
            "full_article": full_article,
        })

        articles_fetched += 1

        if len(valid_articles) >= MAX_VALID_ARTICLES:
            break

    return valid_articles


def save_articles(stock, articles):
    saved_files = []

    for i, article in enumerate(articles):
        filename = f"{article['date']}_{stock}.txt"
        filepath = os.path.join(OUTPUT_DIR, filename)

        # Handle duplicate file names (appending _1, _2, etc.)
        counter = 1
        while os.path.exists(filepath):
            filepath = os.path.join(OUTPUT_DIR, f"{article['date']}_{stock}_{counter}.txt")
            counter += 1

        with open(filepath, "w", encoding="utf-8") as f:
            f.write(f"Stock: {stock}\n")
            f.write(f"Date: {article['date']}\n")
            f.write(f"Title: {article['title']}\n")
            f.write(f"URL: {article['url']}\n")
            f.write(f"Full Article:\n{article['full_article']}\n")

        saved_files.append(filepath)
        print(f"Saved: {filepath}")

    return len(saved_files) > 0


def fetch_recent_ohlc(stock):
    try:
        ticker = yf.Ticker(stock)
        df = ticker.history(period="1d")

        if not df.empty:
            data = {
                "Stock": stock,
                "Open_Price": df.iloc[-1]["Open"],
                "Close_Price": df.iloc[-1]["Close"],
                "High_Price": df.iloc[-1]["High"],
                "Low_Price": df.iloc[-1]["Low"],
                "Volume": df.iloc[-1]["Volume"],
            }
            print(f"OHLC Data Fetched: {stock} - Close: {data['Close_Price']}")
            return data
        else:
            print(f"No OHLC data found for {stock}")
            return None

    except Exception as e:
        print(f"Failed to fetch OHLC data for {stock}: {e}")
        return None


def fetch_past_21_days_ohlc(stock):
    try:
        ticker = yf.Ticker(stock)
        df = ticker.history(period="21d")

        if not df.empty:
            df["Stock"] = stock
            df.reset_index(inplace=True)
            df["Date"] = df["Date"].dt.tz_localize(None)  # Remove timezone
            df = df[["Date", "Stock", "Open", "High", "Close", "Low", "Volume"]]
            return df
        else:
            print(f"No past 21 days OHLC data found for {stock}")
            return pd.DataFrame()

    except Exception as e:
        print(f"Failed to fetch past 21 days OHLC data for {stock}: {e}")
        return pd.DataFrame()


all_ohlc_data = []
past_21_days_ohlc_data = []

for stock in stocks:
    # 1. Scrape and Save News Articles
    articles = scrape_news(stock)
    saved_articles = save_articles(stock, articles) if articles else False

    # 2. Fetch OHLC Data only if news was saved
    if saved_articles:
        ohlc_data = fetch_recent_ohlc(stock)
        if ohlc_data:
            all_ohlc_data.append(ohlc_data)

        # 3. Fetch past 21 days OHLC Data
        past_21_days_df = fetch_past_21_days_ohlc(stock)
        if not past_21_days_df.empty:
            past_21_days_ohlc_data.append(past_21_days_df)

    # Pause between requests
    time.sleep(random.uniform(5, 10))

# Save OHLC Data to Excel
if all_ohlc_data:
    ohlc_df = pd.DataFrame(all_ohlc_data)
    ohlc_df.to_excel("stock_ohlc_data.xlsx", index=False)
    print("\nOHLC data saved to 'stock_ohlc_data_ex.xlsx'")
else:
    print("\nNo OHLC data to save.")

# Save Past 21 Days OHLC Data to Excel
if past_21_days_ohlc_data:
    past_21_days_df = pd.concat(past_21_days_ohlc_data, ignore_index=True)
    past_21_days_df.to_excel("stock_ohlc_past_21_days.xlsx", index=False)
    print("\nPast 21 days OHLC data saved to 'Stock_ohlc_past_21_days.xlsx'")
else:
    print("\nNo past 21 days OHLC data to save.")

print("\nNews scraping and OHLC data fetching completed!")


Fetching news for: SPG...
Saved: news_articles\2025-02-05_SPG.txt
Saved: news_articles\2025-02-05_SPG_1.txt
Saved: news_articles\2025-02-19_SPG.txt
Saved: news_articles\2025-02-16_SPG.txt
Saved: news_articles\2025-02-19_SPG_1.txt
Failed to fetch OHLC data for SPG: Too Many Requests. Rate limited. Try after a while.
Failed to fetch past 21 days OHLC data for SPG: Too Many Requests. Rate limited. Try after a while.

Fetching news for: WELL...
Saved: news_articles\2025-02-12_WELL.txt
Saved: news_articles\2025-02-14_WELL.txt
Saved: news_articles\2025-02-19_WELL.txt
Saved: news_articles\2025-02-19_WELL_1.txt
Saved: news_articles\2025-02-19_WELL_2.txt
Failed to fetch OHLC data for WELL: Too Many Requests. Rate limited. Try after a while.
Failed to fetch past 21 days OHLC data for WELL: Too Many Requests. Rate limited. Try after a while.

No OHLC data to save.

No past 21 days OHLC data to save.

News scraping and OHLC data fetching completed!


In [2]:
import os
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime

analyzer = SentimentIntensityAnalyzer()

def perform_sentiment_analysis(article_text):
    vader_scores = analyzer.polarity_scores(article_text)
    return vader_scores['compound']

def parse_date(date_string):
    formats = ['%Y-%m-%d', '%d-%m-%Y', '%d/%m/%Y']
    for fmt in formats:
        try:
            return datetime.strptime(date_string, fmt)
        except ValueError:
            continue
    print(f"Invalid date format: {date_string}")
    return None

def assign_dynamic_weights(group):
    max_date = group['Date'].max()
    group['Time_Diff_Days'] = (max_date - group['Date']).dt.days

    min_weight = 0.1
    max_weight = 0.9
    decay_factor = 0.5

    group['Weight'] = max_weight * np.exp(-decay_factor * group['Time_Diff_Days'])
    group['Weight'] = group['Weight'].clip(lower=min_weight, upper=max_weight)
    group['Weight'] = group['Weight'] / group['Weight'].sum()

    return group

def process_files(input_folder, output_excel, ohlc_file, final_output_with_ohlc):
    data = []

    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            with open(os.path.join(input_folder, filename), 'r', encoding='utf-8') as file:
                lines = file.readlines()

                stock = date = title = url = None
                full_article_lines = []
                collecting_article = False

                for line in lines:
                    if line.startswith("Stock:"):
                        stock = line.split(":", 1)[1].strip()
                    elif line.startswith("Date:"):
                        date = line.split(":", 1)[1].strip()
                    elif line.startswith("Title:"):
                        title = line.split(":", 1)[1].strip()
                    elif line.startswith("URL:"):
                        url = line.split(":", 1)[1].strip()
                    elif line.startswith("Full Article:"):
                        collecting_article = True
                    elif collecting_article:
                        full_article_lines.append(line.strip())

                full_article = "\n".join(full_article_lines).strip()

                if not full_article or len(full_article.split()) < 5:
                    print(f"Skipping file due to empty or short article: {filename}")
                    continue

                sentiment_score = perform_sentiment_analysis(full_article)

                date = parse_date(date)
                if not date:
                    print(f"Skipping file due to invalid date: {filename}")
                    continue

                if stock and date and title and url:
                    data.append({
                        'Date': date,
                        'Stock': stock,
                        'Title': title,
                        'URL': url,
                        'Sentiment_score': sentiment_score
                    })
                else:
                    print(f"Skipping file due to missing fields: {filename}")

    if data:
        df = pd.DataFrame(data)
        df.sort_values(by=["Stock", "Date"], ascending=[True, False], inplace=True)

        final_results = []

        for stock, group in df.groupby('Stock'):
            group = group.head(5).copy()
            group = assign_dynamic_weights(group)

            weighted_avg_score = (group['Sentiment_score'] * group['Weight']).sum()
            sentiment_std = group['Sentiment_score'].std() or 0

            if weighted_avg_score >= sentiment_std:
                final_label_std = 'Positive'
            elif weighted_avg_score <= -sentiment_std:
                final_label_std = 'Negative'
            else:
                final_label_std = 'Neutral'

            if weighted_avg_score > 0.4:
                final_label_threshold = 'Positive'
            elif weighted_avg_score < -0.4:
                final_label_threshold = 'Negative'
            else:
                final_label_threshold = 'Neutral'

            for _, row in group.iterrows():
                final_results.append({
                    'Date': row['Date'],
                    'Stock': stock,
                    'Title': row['Title'],
                    'URL': row['URL'],
                    'Positive_score': max(row['Sentiment_score'], 0),
                    'Negative_score': abs(min(row['Sentiment_score'], 0)),
                    'Sentiment_score': row['Sentiment_score'],
                    'Weightage': row['Weight'],
                    'Weighted_average_score': weighted_avg_score,
                    'Final_sentiment_label_on_std': final_label_std,
                    'Final_sentiment_label_on_threshold_value': final_label_threshold
                })

        final_df = pd.DataFrame(final_results)
        final_df['Date'] = final_df['Date'].dt.strftime('%d %b %Y %H:%M')
        final_df.to_excel(output_excel, index=False)

        print(f"Output saved to {output_excel}")

        # Load OHLC Data
        ohlc_df = pd.read_excel(ohlc_file)

        # Merge with Analysis Results on 'Stock' only
        merged_df = pd.merge(final_df, ohlc_df, on='Stock', how='left')

        # Fill missing values for each stock by forward and backward fill
        merged_df[['Open_Price', 'Close_Price', 'High_Price', 'Low_Price', 'Volume']] = merged_df.groupby('Stock')[
            ['Open_Price', 'Close_Price', 'High_Price', 'Low_Price', 'Volume']
        ].transform(lambda x: x.ffill().bfill())

        merged_df.to_excel(final_output_with_ohlc, index=False)
        print(f"Final output with OHLC data saved to '{final_output_with_ohlc}'")

    else:
        print("No valid data to process.")

input_folder = 'news_articles'
output_excel = 'filtered_output1.xlsx'
ohlc_file = 'stock_ohlc_data.xlsx'
final_output_with_ohlc = 'final_output_with_ohlc.xlsx'

process_files(input_folder, output_excel, ohlc_file, final_output_with_ohlc)

Output saved to filtered_output1.xlsx
Final output with OHLC data saved to 'final_output_with_ohlc.xlsx'


In [3]:
import pandas as pd
import numpy as np
import talib

# Load your data
df = pd.read_excel('stock_ohlc_past_21_days.xlsx')

# Ensure Date is datetime type and sort by Stock and Date
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by=['Stock', 'Date'], inplace=True)

# Indicators, Swing Highs/Lows, Stop Loss, Take Profit per stock
def process_stock(stock_df):
    # Indicators
    stock_df['SMA9'] = talib.SMA(stock_df['Close'], timeperiod=9)
    stock_df['SMA21'] = talib.SMA(stock_df['Close'], timeperiod=21)
    stock_df['RSI14'] = talib.RSI(stock_df['Close'], timeperiod=14)

    # Buy/Sell Signal
    stock_df['Buy_Signal'] = (stock_df['SMA9'] > stock_df['SMA21']) & (stock_df['RSI14'] > 50)
    stock_df['Sell_Signal'] = (stock_df['SMA9'] < stock_df['SMA21']) & (stock_df['RSI14'] < 50)

    # Detect Swing Highs/Lows
    def detect_swing_high(stock_df, window=3):
        stock_df['Swing_High'] = np.nan
        for i in range(window, len(stock_df) - window):
            if stock_df['High'].iloc[i] == max(stock_df['High'].iloc[i - window: i + window + 1]):
                stock_df.loc[stock_df.index[i], 'Swing_High'] = stock_df['High'].iloc[i]
        return stock_df

    def detect_swing_low(stock_df, window=3):
        stock_df['Swing_Low'] = np.nan
        for i in range(window, len(stock_df) - window):
            if stock_df['Low'].iloc[i] == min(stock_df['Low'].iloc[i - window: i + window + 1]):
                stock_df.loc[stock_df.index[i], 'Swing_Low'] = stock_df['Low'].iloc[i]
        return stock_df

    stock_df = detect_swing_high(stock_df)
    stock_df = detect_swing_low(stock_df)

    # Fill NAs in Swing High/Low
    stock_df['Swing_High'] = stock_df['Swing_High'].ffill()
    stock_df['Swing_Low'] = stock_df['Swing_Low'].ffill()

    # Add Stop Loss based on Swing Levels
    stock_df['Buy_Stop_Loss'] = stock_df.apply(lambda x: x['Swing_Low'] if x['Buy_Signal'] else np.nan, axis=1)
    stock_df['Sell_Stop_Loss'] = stock_df.apply(lambda x: x['Swing_High'] if x['Sell_Signal'] else np.nan, axis=1)

    # Risk-to-Reward Ratio for Take Profit
    RRR = 2  # Example: Risk-to-Reward Ratio of 2:1

    # Add Take Profit levels
    stock_df['Buy_Take_Profit'] = stock_df.apply(
        lambda x: x['Close'] + (x['Close'] - x['Buy_Stop_Loss']) * RRR if x['Buy_Signal'] else np.nan, axis=1
    )
    stock_df['Sell_Take_Profit'] = stock_df.apply(
        lambda x: x['Close'] - (x['Sell_Stop_Loss'] - x['Close']) * RRR if x['Sell_Signal'] else np.nan, axis=1
    )

    return stock_df

# Group by Stock and process each stock separately
df = df.groupby('Stock', group_keys=False).apply(process_stock)

# Extract the most recent data for each stock
latest_data = df.loc[df.groupby('Stock')['Date'].idxmax()]

# Save full data and latest data
df.to_excel('output_with_swing_high_low_take_profit.xlsx', index=False)
latest_data.to_excel('output_most_recent_data.xlsx', index=False)

print("Processing completed successfully!")

C:\Users\91815\AppData\Local\Temp\ipykernel_15872\3263989602.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Stock', group_keys=False).apply(process_stock)


Processing completed successfully!


In [4]:
import pandas as pd

# Load the final sentiment analysis output
analyzer_df = pd.read_excel('final_output_with_ohlc.xlsx')

# Load the most recent stock data with indicators
indicators_df = pd.read_excel('output_most_recent_data.xlsx')

# Select only necessary columns from indicators data
indicator_columns = [
    'Stock', 'SMA9', 'SMA21', 'RSI14', 'Buy_Signal', 'Sell_Signal',
    'Swing_High', 'Swing_Low', 'Buy_Stop_Loss', 'Sell_Stop_Loss',
    'Buy_Take_Profit', 'Sell_Take_Profit'
]

indicators_df = indicators_df[indicator_columns]

# Merge the two dataframes based on 'Stock'
merged_df = pd.merge(analyzer_df, indicators_df, on='Stock', how='left')

# Save the merged dataframe to a new Excel file
merged_df.to_excel('final_output_with_indicators.xlsx', index=False)

print("Final output with indicators merged successfully!")


Final output with indicators merged successfully!
